![title](https://cdn-images-1.medium.com/max/1200/1*i1d88Q8NNrRv6kjf7Ssw4g.png)

# Introducción a Redis


Se trata de una base de datos clave valor con las siguientes características:

- Muy rápida: 110000 sets por segundo y alrededor de 81000 gets
- Soporta estructuras de datos complejas en el valor
- Operaciones atómicas --> consistencia
- Escalable (admite clúster)
- Pensada para funcionar en memoria, pero admite persistencia

Vamos a utilizar redis desde su interfaz con Python. Para detalles de este interfaz consultar 
https://redis-py.readthedocs.io/en/latest/

Para una buena introducción a los tipos de Redis y sus operaciones
https://redis.io/topics/data-types-intro

Para una lista de comandos más útiles:

https://gist.github.com/Yogendra0Sharma/3f83aa825470ee989b28a5314bd5a4a6

### Instalación

En esta práctica vamos a utilizar redis en red (ver instrucciones aparte), por lo que lo único de que debeos disponer es de Anaconda y de la librería correspondiente. Para ello, desde Anaconda Prompt hacer

```
pip install redis
```

Si alguien prefiere instalarlo en Linux, es habitual hacerlo a partir del fuente (es muy ligero)

```
wget http://download.redis.io/redis-stable.tar.gz
tar xvzf redis-stable.tar.gz
cd redis-stable
make
make test
```

para dejarlo instalado

```
sudo make install
```

también se puede hacer directamente:

```
sudo apt-get update 
sudo apt-get install redis-server
```

### Iniciar redis
- Arquitectura cliente-servidor 
- Arranque del servidor

```
redis-server
```

o con una configuración

```
redis-server conf
```

Si estamos conectando a un servidor en la nube no habrá que hacer esto.

### Comprobar si funciona

Desde la línea de comandos

```
redis-cli ping
PONG
```

Desde Python:


In [1]:
# Ojo, cambiar estos datos por los de vuestro acceso a red en redislabs

redisconexion = "redis-12588.c135.eu-central-1-1.ec2.cloud.redislabs.com"  # será algo de la forma "...ec2.cloud.redislabs.com"
redispuerto = 12588 # cambiar solo si es diferente
passwd = "YNUBRlaFh6kJnTKfDxsXur44M3jOkNqy" # poner el passwd correspondiente

import redis
r = redis.Redis(host=redisconexion, password=passwd, port=redispuerto, charset="utf-8", decode_responses=True, db=0)
r.ping() # debe mostrar True

True

### Asignación clave valor

Las operaciones básicas en toda base de datos clave valor: *set* y *get*

In [6]:
r.set('nombre:usuario', 'Bertoldo')

True

In [7]:
x = r.get('nombre:usuario')
x

'Bertoldo'

Redis permite almacenar distintos tipos de datos, no solo strings:

In [8]:
r.set('edad:usuario',27)
x = int(r.get('edad:usuario'))
x,type(x)

(27, int)

Es importante notar que estamos trabajando en memoria, aunque los datos se pueden grabar a disco (veremos como).

Si queremos borrar todas las claves de memoria:

In [9]:
r.dbsize()

2

In [10]:
r.flushall()
r.dbsize()

0

### Set en datalle

La sintaxis completa de set en Python es

```[Python]
set(name, value, ex=None, px=None, nx=False, xx=False)
```
donde
- ex establece un tiempo de vida en segundos
- px establece un tiempo de vida en milisegundos
- nx vale True si se quiere que se cambie el valor solo si la clave es nueva 
- xx vale True si se quiere que se cambie el valor solo si la clave ya existe


**P** ¿Qué valor tendra "x" tras ejecutar este código? y si se cambia el 5 por un 10?


In [12]:
import time
r.set("x",10,ex=8)
time.sleep(5)
r.set("x",-1,xx=True)
r.get("x")


'-1'

Para saber el tiempo que le queda de vida a una clave podemos utilizar el *time to live*

In [13]:
r.set("x",10,ex=5)
time.sleep(3)
r.ttl("x")

2

Para alargar la vida podemos utilizar *expire* (o *pexpire* para milisegundos)

In [14]:
r.set("x",10,ex=5)
time.sleep(3)
print(r.ttl("x"))
r.expire("x",-3)
print(r.ttl("x"))
time.sleep(3)
print(r.ttl("x"))
r.get("x")

2
-2
-2


TTL devuelve el tiempo en segundos de vida, -2 si la clave no existe o -1 si sí existe pero es permanente

Para quitarle la condena a muerte podemos utilizar *persist*

In [15]:
r.set("x",10,ex=5)
time.sleep(3)
print(r.ttl("x"))
r.persist("x")
print(r.ttl("x"))
print(r.ttl("y"))


1
-1
-2


Aunque *ttl* toma el valor -2 cuando la clave no existe, es mejor utilizar *exists*

In [16]:
print(r.exists("x"))
print(r.exists("y"))

1
0


Además podemos eliminar una clave con  *delete*  (*del* en la  consola)

In [17]:
r.delete("x")
print(r.exists("x"))

0


### Cadenas

En el caso de cadenas de caracteres (strings), Redis ofrece las funciones 

- append(clave,s), que añade una cadena s al final del valor asociado a clave. Devuelve el número de caracteres que tiene la cadena
- strlen(clave): la longitud de una cadena
- getrange(clave,begin,end): devuelve un fragmento de una cadena desde el caracter *begin* hasta el caracter *end*
- setrange(clave,begin,s): reemplaza el contenido de la cadena a partir del caracter *begin* por *s*. no inserta sino que machaca el resultado.

Estas operaciones son muy rápidas O(1) (sin tener en cuenta el tiempo de copiar la cadena. si sí lo tenemos en cuenta O(M))

In [47]:
r.flushdb()
r.set("saludo", "Hola ")
r.append("saludo"," Mundo")
r.get("saludo")

'Hola  Mundo'

In [48]:
l = r.strlen("saludo")
r.getrange("saludo", 5, l)

' Mundo'

In [49]:
r.setrange("saludo",5,"Chung")
r.get("saludo")

'Hola Chungo'

Una aplicación muy típica de las cadenas en Redis es la captura de datos desde fuentes streaming.

**Ejemplo**
Ejecutar en un otra pestaña el notebook *sensor*


In [21]:
r.get("sensor")

'   320.9'

In [22]:
s = str(r.get("sensor"))
r.set("sensordata",s)

True

**P2** Escribir un código en Python que, mientras que la longitud (strlen) de la clave "sensordata" sea menor que 1000 vaya añadiendo (append) a la clave *sensordata* cada nuevo valor leído del sensor (anteponer ':' para distinguir cada clave). Ojo: añadir una nueva lectura solo cuando sea nueva (sea distinta de la última añadida).

Salida esperada:
```
36950.9: 62610.9: 62620.6: 62630.4: 62640.4: 62650.0: 62660.2: 62670.6: 62680.7: 62690.2: 62700.8: 62710.1: 62730.2: 62740.0: 62750.9: 62760.2: 62770.2: 62780.9: 62790.8: 62800.0: 62810.1: 62820.5: 62830.4: 62840.6: 62850.2: 62860.8: 62870.6: 62880.1: 62890.0: 62900.3: 62910.3: 62920.1: 62930.0: 62940.9: 62950.4: 62960.2: 62970.0: 62980.5: 62990.2: 63000.9: 63010.7: 63020.0: 63030.6: 63040.6
```

In [11]:
# Solución. Recordar utilizar el notebook sensor 
r.delete("sensordata")
v = ""
while r.strlen("sensordata")<1000:
    nv = r.get("sensor")
    if nv!=v:
        r.append("sensordata",":"+nv)
        v = nv

print(r.get("sensordata"))

:  1510.1:  1530.6:  1550.0:  1570.9:  1590.3:  1620.9:  1640.6:  1660.4:  1690.3:  1710.8:  1730.5:  1750.1:  1770.2:  1800.0:  1820.5:  1840.1:  1860.8:  1890.2:  1910.3:  1930.4:  1950.1:  1970.8:  2000.6:  2020.0:  2040.5:  2060.3:  2090.9:  2110.1:  2130.5:  2150.6:  2170.8:  2200.3:  2220.3:  2240.9:  2270.8:  2290.8:  2310.2:  2330.1:  2350.2:  2380.7:  2400.2:  2420.5:  2440.9:  2470.9:  2490.9:  2510.6:  2540.2:  2560.8:  2580.8:  2610.0:  2630.8:  2650.9:  2670.2:  2700.8:  2720.3:  2740.0:  2760.3:  2790.8:  2810.7:  2830.6:  2850.3:  2880.5:  2900.6:  2920.5:  2940.5:  2970.5:  2990.0:  3010.2:  3030.4:  3060.6:  3080.3:  3100.1:  3120.0:  3150.6:  3170.3:  3190.4:  3220.1:  3240.8:  3260.2:  3290.8:  3310.3:  3330.6:  3350.9:  3370.6:  3390.7:  3410.0:  3440.2:  3460.0:  3480.4:  3500.6:  3520.6:  3550.3:  3570.2:  3590.5:  3610.8:  3630.1:  3660.3:  3680.6:  3700.9:  3720.5:  3750.3:  3770.3:  3790.9:  3820.1:  3840.9:  3860.7:  3880.6:  3910.9:  3930.8:  3950.9:  3980.7:

Este código tiene un defecto: mezcla el tiempo con el resultado. Si solo estamos interesados en almacenar el dato del sensor y no el tiempo, o queremos hacer ambas cosas por separado, esto nos obligar a 'parsear' el valor en "sensor" para sacar sus componentes.

**P3** ¿No sería mejor hacer 
```[Python] 
        r.set("sensorTime",t)
        r.set("sensorValue",str(randint(0, 9))) 
```
y algo similar en nuestro programa al hacer get? ¿Esto plantearía algún problema?


Para lograr poner varios valores sin perder atomicidad, se puede utilizar mset y mget

In [23]:
r.flushdb() # ejecutar tras este el notebook sensor2
t = -1 # tiempo irreal para coger el primer valor
s = 0 # longitud de sensordata
while r.strlen("sensordata") < 300:
   
    # datos del sensor en una transacción atómica
    v =  r.mget(['sensorTime', 'sensorValue'])
    sensorTime = v[0]
    sensorValue = v[1]
    if sensorTime!=None and int(sensorTime)!=t:
        t=int(v[0]);
        r.append("sensordata",":"+str(v[1]))

    # para la siguiente iteración
    s = r.strlen("sensordata")
    if s%100==0:
        print(s,end=" ")
print("\n",r.get("sensordata"))

100 200 

KeyboardInterrupt: 

Si lo que queremos es obtener el valor de una clave y modificarlo asegurándonos de que no ocurre nada entre tanto, en lugar de 
```[Python]
v = r.get(name)
s.set(name,newvalue)
```
debemos hacer
```[Python]
v = r.getset(name,newvalue)
```


### Enteros
Los enteros admiten algunas operaciones sencillas como son: 
- incr(name, amount=1): incrementa el valor *name* en la cantidad indicada, que es 1 por defecto
- decr(name, amount=1): decrementa el valor *name* en la cantidad indicada, que es 1 por defecto
- incrbyfloat(name, amount=1.0): análogo pero para sumar valores en coma flotante 
- decrbyfloat(name, amount=1.0): análogo pero para sumar valores en coma flotante 

**P4** ¿*incr* no sería equivalente a 

```[Python]
v = r.get(name)
s.set(name,v+1) ?
```



**P5** A partir de sensor2.py hacer un programa que
- Tome 200 valores del sensor
- Nos indique cuantos valores han sido 0 y cuántos han sido 9. Queremos hacerlo con Redis, utilizando incr porque puede haber otros clientes colaborando en la tarea para otros sensores

In [18]:
# Solución. 
r.flushdb() # ejecutar tras este el notebook sensor2
r.set("ceros",0)
r.set("nueves",0)

t = -1 # tiempo irreal para coger el primer valor
for i in range(200):
    v =  r.mget(['sensorTime', 'sensorValue'])
    sensorTime = v[0]
    sensorValue = v[1]
    if sensorTime!=None and int(sensorTime)!=t:
        t=int(v[0]);
        valor = int(v[1])
        if valor==0:
            r.incr("ceros")
        elif valor==9:
            r.incr("nueves")
            
print(r.mget(["ceros","nueves"]))

['14', '12']


## Estructuras compuestas

Son las siguientes:
- Listas
- Streams
- Sets
- Sorted Sets 
- Hashes

Los tipos compuestos o agregados en Redis siguen las siguientes 3 reglas básicas que conviene conocer:

1. Cuando *añadimos* un elemento a un tipo agregado y la clave no existe, se genera de forma automática el valor vacío y a continuación se añade el elemento.
2. Al *eliminar* elementos a un tipo de datos agregado y este queda vacío, la clave asociada es destruida automáticamente (con la excepción de los elementos de tipo Stream)
3. Una llamada a una *operación de solo lectura* como *llen* (longitud de una lista) o de eleminación como *rpop* sobre una clave que no existe, se comporta como si la clave existiera conteniendo el tipo agregado vacío.







### Listas 

Las listas en Redis son *listas enlazadas*, no como arrays (que es el caso de Python). En particular esto significa que:

- las inserciones al principio (*lpush*) o al final (*rpush*) se hacen en tiempo constante.
- El acceso mediante índice es más lento

Utilidades:
- Apuntar los cambios que realiza un usuario (por ejemplo los últimos tweets de un usuario)
- Generar aplicaciones publicor/subscriptor

¿Qué devuelve este código? ¿por qué?

In [4]:
r.delete("lista")
for i in range(10):
    r.lpush("lista",i)
r.get("lista")

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [16]:
r.delete("lista")
for i in range(10):
    r.lpush("lista",i)
print(r.lrange("lista",0,r.llen("lista")))
e = r.rpop("lista")
print(e)
print(r.lrange("lista",0,-1))  # -1 hasta el final
print(r.llen("lista"))
print(r.lindex("lista", 2))
# index("2")

['9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
0
['9', '8', '7', '6', '5', '4', '3', '2', '1']
9
7


El problema es que una lista puede llegar a crecer demasiado, por eso nos interesa crear 'capped lists', utilizando parejas de operaciones *lpush* y *ltrim* (que corta una lista)

In [131]:
r.lpush("lista",*[10,20,30,40,50]) # El * es un truco de python

r.lpush("lista",*[110,120,130,140,150,160])
r.ltrim("lista",0,10)
print(r.lrange("lista",0,-1))

['160', '150', '140', '130', '120', '110', '50', '40', '30', '20', '10']


La siguiente aplicación (productor/consumidor) se puede lograr con lpush y rpop. Si embargo esto tiene el problema de que puede no haber nada, y el rpop devolver None (*polling*). Para resolver esta situación, Redis propone dos funciones *brpop(keys, timeout=0)* y *blpop(keys, timeout=0)*, que quedan esperando (hasta un timeout) para devolver el resultado.

**P7** 
Cambiar el programa sensor2.py para que los valores se vayan añadiendo a una lista *lsensor* con lpush que en ningún caso puede exceder los 10 elementos. Además el tiempo de espera se hará variable, entre 1 y 5 segundos

El consumidor (que añadiremos en lasiguiente caja de código) debe estar leyendo para contar 0s y 9s. Cada lectura (brpop) debe esperar un máximo de 3 segundos, parando en caso contrario.

Consumidor:

In [25]:
# Solución
r.set("ceros",0)
r.set("nueves",0)
SIGUE = True
while SIGUE:
    x  = r.blpop("lsensor",3)  # espera durante 3 segundos
    if x!=None:
        l,valor = x
        if valor=="0":
            r.incr("ceros")
        elif valor=="9":
            r.incr("nueves")
    else:
        SIGUE=False
print(r.mget(["ceros","nueves"]))

['2', '1']


### Hashes

Un *hash* representa un diccionario Python. El nombre de la clave principal agrupa, a su vez, a varios pares clave valor

In [18]:
r.hset("usuarios", "039", "Bertoldo")

r.hset("usuarios", "511", "Herminia")

r.hset("usuarios", "011", "Calixta")

id="511"
print("El nombre del cliente con identificador ",id," es ",r.hget("usuarios",id))

print("El hash tiene", r.hlen("usuarios")," elementos")
print("Claves: ",r.hkeys("usuarios"))
print("Valores: ",r.hvals("usuarios"))
print(r.hdel("usuarios", "039"))
print(r.hgetall("usuarios"))

El nombre del cliente con identificador  511  es  Herminia
El hash tiene 3  elementos
Claves:  ['511', '011', '039']
Valores:  ['Herminia', 'Calixta', 'Bertoldo']
1
{'511': 'Herminia', '011': 'Calixta'}


- hset devuelve 1 si la clave es nuevo, 0 en otro caso. Si se quiere crear la clave solo si no existe, utilizar hsetnx

Otras funciones hash:

- hdel(name, ```*```keys): elimina las claves del hash
- hexists(name, key): indica si la clave existe en el hash
- hgetall(name): devuelve el hash completo en forma de diccionario
- hincrby(name, key, amount=1): incrementa un elemento del hash
- hincrbyfloat(name, key, amount=1.0): lo mismo para float
- hmget(name, keys, ```*```args): get múltiple
- hmset(name, mapping): set múltiple, *mapping* es un diccionario

**P8** 
Cambiar el consumidor de P7 para que cuente el número de apariciones de cada valor emitido por el sensor, usando un hash "valores" que irá añadiendo como nueva clave y valor 1 los nuevos valores emitidos por el sensor, o incrementando los valores ya existentes

In [142]:
# Solución

r.delete("valores")    
SIGUE=True
while SIGUE:
    v = r.blpop("lsensor",3)
    if v!=None:
        s,a = v
        r.hincrby("valores",a)
        print(a)
    else:
        SIGUE=False
        
# mostrar el resultado
for i in range(10):
    print(i,r.hget("valores",i))


7
7
1
2
2
0
8
3
9
6
7
0 1
1 1
2 2
3 1
4 None
5 None
6 1
7 3
8 1
9 1


### Sets

Colecciones no ordenadas de strings sin repeticiones.

- sadd(name,```*```e): añade un elemento al set
- smembers(name): elementos en el set
- scard(name): cardinal
- sismember(name,e): Indica si e está en el conjunto (True) o no (False)


In [9]:
r.sadd("s",*[1,2,3])
r.sadd("s",3)

print(r.scard("s"))
print(r.smembers("s"))

print(r.sismember("s",4))


3
{'2', '3', '1'}
False


**P9**: Queremos utizar los conjuntos para asociar tags a distintos elementos o items. Completar el siguiente código

In [143]:
def addTag(r,item,tag):
    # queremos que en el conjunto asociado al item se añada el tag, y al revés, 
    # en el del tag el item
    r.sadd(item,tag)
    r.sadd(tag,item)
    
    

In [16]:
# Para probar
r.flushall()
addTag(r,"Spiderman","fantasía")
addTag(r,"Spiderman","superhéroes")
addTag(r,"Spiderman","comic")
addTag(r,"Spiderman","arañas")
addTag(r,"Peppa Pig","fantasía")
addTag(r,"Peppa Pig","infantil")
addTag(r,"Peppa Pig","charcos")

# queremos saber todas las etiquetas asociadas a Spiderman
print(r.smembers("Spiderman"))
# todos los items asociados al tag "fantasía"
print(r.smembers("fantasía"))


{'comic', 'superhéroes', 'fantasía', 'arañas'}
{'Spiderman', 'Peppa Pig'}


Los conjuntos también incluyen las típicas operaciones de unión, intersección, etc-

In [19]:
# Valores en común
r.sinter("Spiderman","Peppa Pig")

{'fantasía'}

Una operación curiosa es spop(name) que elimina y devuelve un elemento al azar del conjunto.

**P10** Dada la siguiente definición de baraja, escribir un código que simule repartir 5 cartas a dos jugadores (5 a cada uno). Llamemos c1 y c2 a estos dos conjuntos


In [146]:
r.delete("mazo")
r.sadd("mazo",*[i+j for i in ["C","P","T","D"]
                    for j in ["1","2","3","4","5","6","7","8","9","10","J","Q","K"]])
r.smembers("mazo")

{'C1',
 'C10',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'CJ',
 'CK',
 'CQ',
 'D1',
 'D10',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'DJ',
 'DK',
 'DQ',
 'P1',
 'P10',
 'P2',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
 'PJ',
 'PK',
 'PQ',
 'T1',
 'T10',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'TJ',
 'TK',
 'TQ'}

In [148]:
# solución
r.delete("c1")
r.delete("c2")

for i in range(5):
    r.sadd("c1",r.spop("mazo"))
    r.sadd("c2",r.spop("mazo"))


In [149]:
print(r.smembers("c1"))
print(r.smembers("c2"))
print(r.smembers("mazo"))


{'PQ', 'C8', 'D4', 'T6', 'T3'}
{'C7', 'C1', 'D8', 'P4', 'T7'}
{'CQ', 'D7', 'TQ', 'PJ', 'P6', 'CK', 'T2', 'C3', 'P1', 'D5', 'D6', 'T10', 'C2', 'D1', 'DK', 'P5', 'T1', 'DJ', 'P2', 'P10', 'P8', 'DQ', 'C6', 'P9', 'T8', 'C5', 'D2', 'D3', 'T5', 'PK', 'TJ', 'D10', 'C4', 'P3', 'D9', 'CJ', 'TK', 'C10', 'P7', 'T4', 'C9', 'T9'}


### Sorted Sets

Los conjuntos ordenados son un tipo muy importante en Redis. Se parecen a los conjuntos con un añadido: cada cadena del conjunto tiene un *score* (número real), y el conjunto está ordeado por este valor según el siguiente criterio:

- Si A y B son dos elementos del conjunto y A.score > B.score, entonces A>B
- Si A.score==B.score entonces A > B si A es lexicográficamente mayor que B

**Ejemplo** Guardamos usuarios de Instagram con su número de seguidores


In [150]:
r.delete("instagram")
r.zadd("instagram",{"Ariana": 149, "Cristiano":158, "Selena":147, "Dwayne Johnson":135, "Kim Kardashian":131})
print(r.zcard("instagram"))
print(r.zrange("instagram",0,-1))
print(r.zrevrange("instagram",0,-1))
print(r.zrange("instagram",0,-1, withscores=True))
print(r.zrangebyscore("instagram",130,150, withscores=True))
print(r.zrank("instagram","Cristiano"))
print(r.zscore("instagram","Cristiano"))



5
['Kim Kardashian', 'Dwayne Johnson', 'Selena', 'Ariana', 'Cristiano']
['Cristiano', 'Ariana', 'Selena', 'Dwayne Johnson', 'Kim Kardashian']
[('Kim Kardashian', 131.0), ('Dwayne Johnson', 135.0), ('Selena', 147.0), ('Ariana', 149.0), ('Cristiano', 158.0)]
[('Kim Kardashian', 131.0), ('Dwayne Johnson', 135.0), ('Selena', 147.0), ('Ariana', 149.0)]
4
158.0


También hay funciones para modificar los zsets:

- zincrby(name, amount, value): incrementa value en amount dentro del zset name
- zpopmax(name, count=None): devuelve el elemento con máximo score y lo quita del zset
- zpopmin(name, count=None):devuelve el elemento con mínimo score y lo quita del zset
- zrem(name, ```*```values): elimina los elementos values del zset

### Bases de datos

Hasta ahora todo lo hemos hecho en la misma base de datos. Sin embargo, podemos distinguir entre diferentes bases de datos, para agrupar claves en temáticas similares. En Redis se utilizan números: 0 para la primera base de datos, 1 para la segunda, etc.

```
r3 = redis.Redis(host='localhost', port=6379, db =3)
r3.ping()
```

- move(name, db); cambia la clave name a la base de datos db
- swapdb(first, second): intercambia dos bases de datos, de forma que todo el mundo conectado a *first* pasa a ver *second* y viceversa. Ejemplo: *r.swapdb(0,1)*

### Persistencia


Dos formas:
- RDF: Grabación a los intervalos indicados
- AOF: Se va guardando cada operación en un log

**RDB**
Ventajas:
- Perfecto para backups
- Recuperaciones más rápidas

Desventajas:
- Puede perder datos
- En los momentos de la grabación el servidor puede detenerse momentáneamente si la base de datos es muy grande


**AOF**
Ventajas:
- Grabación constante, recuperación completa
- Se puede revisar la secuencia de operaciones (se crea un log)

Desventajas:
- Ficheros de grabación más grandes
- En casos extremos puede ser más lento.
- Para la recuperación hay que reconstruir la base de datos paso a paso.

La forma elegida se puede cambiar en el fichero de configuración inicial.

- lastsave(): la última vez que se grabó la base de datos
- save(): graba la base de datos de forma persistente
- bgsave(): igual pero en background, sin bloquear



In [154]:
## ejemplo de grabado
r.flushall()
r.set("a",1)
r.save()
r.set("b",1)


True

Matar el servidor
```
ps -A | grep "redis"
3733 ?        00:04:25 redis-server

kill -9 3733
redis-server
```

In [152]:
print(r.get("a"))
print(r.get("b"))


1
1
